In [ ]:
import pandas as pd

In [ ]:
pib_br = pd.read_csv("PIB 1995-2024.csv", sep=",")
pib_br = pd.DataFrame(pib_br)
pib_br['Data'] = pib_br['Data'].astype(str)
pib_br['Data'] = pib_br['Data'].str.replace('.', '/')
pib_br['Data'] = pd.to_datetime(pib_br['Data'], format= '%Y/%m')
pib_br = pib_br.rename(columns={'PIB - R$ (milhões)  - Banco Central do Brasil, Boletim, Seção Atividade Econômica (Bacen / Boletim / Ativ. Ec.) - BM12_PIB12 - ':'PIB'})
pib_br['PIB'] = pd.to_numeric(pib_br['PIB'], errors='coerce')
pib_br['PIB'] = pib_br['PIB'].apply(lambda x: '{:.2f}'.format(x))

In [ ]:
pib_usa= pd.read_csv("GDP 1992-2024.csv", sep=",")
pib_usa=pd.DataFrame(pib_usa)
pib_usa=pib_usa.rename(columns={'Unnamed: 0':'Data'})

pib_usa=pib_usa[['Data', 'Monthly Nominal GDP Index']]
pib_usa=pib_usa.dropna()

pib_usa['Data'] = pib_usa['Data'].str.replace(' - ', '/')
pib_usa['Data'] = pib_usa['Data'].str.replace(' ', '')
meses = {
  'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 
  'May': '05', 'June': '06', 'July':'07', 'Jun': '06', 'Jul': '07', 'Aug': '08', 
  'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12',
}
for mes in meses.keys():
  pib_usa['Data'] = pib_usa['Data'].str.replace(mes, meses[mes])

pib_usa['Data'] = pd.to_datetime(pib_usa['Data'], format='%Y/%m')
pib_usa = pib_usa.rename(columns={'Monthly Nominal GDP Index':'PIB'})
pib_usa['PIB'] = pd.to_numeric(pib_usa['PIB'], errors='coerce')
pib_usa['PIB'] = pib_usa['PIB'].apply(lambda x: '{:.2f}'.format(x))

In [ ]:
ibov = pd.read_csv("BVSP 1993-2024.csv", sep=",")
ibov = pd.DataFrame(ibov)
ibov['Date'] = ibov['Date'].astype(str)
ibov['Date'] = ibov['Date'].str.replace('-', '/')
ibov['Date'] = pd.to_datetime(ibov['Date'], format='%Y/%m/%d')
ibov = ibov.dropna()
ibov.set_index('Date', inplace=True)
ibov = ibov.resample('MS').first()
ibov.reset_index(inplace=True)
ibov = ibov[['Date', 'Close']]
ibov = ibov.rename(columns={'Date':'Data'})
ibov = ibov.rename(columns={'Close':'Fechamento'})
ibov['Fechamento'] = pd.to_numeric(ibov['Fechamento'], errors='coerce')
ibov['Fechamento'] = ibov['Fechamento'].apply(lambda x: '{:.2f}'.format(x))

In [ ]:
sp500 = pd.read_csv("SP500 1993-2024.csv", sep=",")
sp500 = sp500.dropna()
sp500 = sp500[['Data','Fechamento*']]
sp500['Data'] = sp500['Data'].astype(str)
sp500['Data'] = sp500['Data'].str.replace(' de ', '/').str.replace(' ', '').str.replace('.', '')
meses = {
  'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 
  'mai': '05', 'junho': '06', 'julho': '07', 'jun': '06', 'jul': '07', 'ago': '08', 
  'set': '09', 'out': '10', 'nov': '11', 'dez': '12',
}
for mes in meses.keys():
  sp500['Data'] = sp500['Data'].str.replace(mes, meses[mes])
sp500['Data'] = pd.to_datetime(sp500['Data'], format='%d/%m/%Y')
sp500['Fechamento*'] = sp500['Fechamento*'].str.replace('.', '').str.replace(',', '.')
sp500 = sp500.rename(columns={'Fechamento*':'Fechamento'})
sp500['Fechamento'] = pd.to_numeric(sp500['Fechamento'], errors='coerce')
sp500['Fechamento'] = sp500['Fechamento'].apply(lambda x: '{:.2f}'.format(x))

In [29]:
pib_br

,Data,PIB
0,1995-01-01,47028.60
1,1995-02-01,49954.50
2,1995-03-01,60024.30
3,1995-04-01,57789.50
4,1995-05-01,56350.50
...,...,...
348,2024-01-01,874397.90
349,2024-02-01,885318.10
350,2024-03-01,954200.80
351,2024-04-01,953853.00


In [30]:
pib_usa

,Data,PIB
0,1992-01-01,6315.73
1,1992-02-01,6356.51
2,1992-03-01,6417.08
3,1992-04-01,6443.26
4,1992-05-01,6433.81
...,...,...
380,2023-09-01,27819.28
381,2023-10-01,27740.33
382,2023-11-01,27952.75
383,2023-12-01,28177.92


In [31]:
ibov

,Data,Fechamento
0,1993-04-01,24.50
1,1993-05-01,24.10
2,1993-06-01,37.20
3,1993-07-01,52.30
4,1993-08-01,62.70
...,...,...
370,2024-02-01,128481.00
371,2024-03-01,129180.00
372,2024-04-01,126990.00
373,2024-05-01,127122.00


In [32]:
sp500

,Data,Fechamento
0,2024-06-28,5460.48
1,2024-06-01,5460.48
2,2024-05-01,5277.51
3,2024-04-01,5035.69
4,2024-03-01,5254.35
...,...,...
370,1993-09-01,458.93
371,1993-08-01,463.56
372,1993-07-01,448.13
373,1993-06-01,450.53
